In [1]:
import pandas as pd

In [2]:
usuarios = pd.read_csv("data/usuarios.csv")
clientes = pd.read_csv("data/clientes.csv")
produtos = pd.read_csv("data/produtos.csv")
vendas = pd.read_csv("data/vendas.csv")
itens_venda = pd.read_csv("data/itens_venda.csv")

In [14]:
colClientes = ["id_cliente", "regiao", "canal_aquisicao"]
colItens = ["id_venda", "id_produto", "quantidade", "valor_total"]
colVendas = ["id_venda", "data_venda", "id_cliente"]

In [10]:
print(itens_venda.columns)

Index(['id_item_venda', 'id_venda', 'id_produto', 'quantidade', 'valor_total'], dtype='object')


In [15]:
bat1 = pd.merge(vendas[colVendas], itens_venda[colItens], on="id_venda", how="left")


In [18]:
# display(bat1)
display(bat1[bat1['id_venda'] == 199])
display(vendas[vendas['id_venda'] == 199])

,id_venda,data_venda,id_cliente,id_produto,quantidade,valor_total
378,199,2025-06-07,178,3,1,50000.0
379,199,2025-06-07,178,2,1,100000.0
380,199,2025-06-07,178,4,1,100000.0


,id_venda,data_venda,id_cliente,id_usuario,valor_total_venda
198,199,2025-06-07,178,4,250000.0


In [1]:
import pandas as pd
import os
from datetime import datetime

# Carrega todos os datasets
df_usuarios = pd.read_csv("data/usuarios.csv")
df_clientes = pd.read_csv("data/clientes.csv")
df_produtos = pd.read_csv("data/produtos.csv")
df_vendas = pd.read_csv("data/vendas.csv")
df_itens = pd.read_csv("data/itens_venda.csv")

# --- 1. Preparação e Unificação dos Dados ---

# Converte colunas de data para o formato datetime
df_vendas['data_venda'] = pd.to_datetime(df_vendas['data_venda'])

# Junta (merge) todas as tabelas para criar uma visão completa
# Usamos os campos de ID para conectar as informações
df_completo = pd.merge(df_itens, df_vendas, on='id_venda')
df_completo = pd.merge(df_completo, df_produtos, on='id_produto')
df_completo = pd.merge(df_completo, df_usuarios, on='id_usuario')
df_completo = pd.merge(df_completo, df_clientes, on='id_cliente')

# --- 2. Criação de Colunas para Análise e Filtros ---

# Extrai ano, mês e trimestre da data da venda
df_completo['ano'] = df_completo['data_venda'].dt.year
df_completo['mes'] = df_completo['data_venda'].dt.month
df_completo['trimestre'] = df_completo['data_venda'].dt.quarter
# --- 3. Agregação dos Dados ---

# Define as colunas pelas quais queremos agrupar os dados
colunas_agrupamento = [
    'ano', 'trimestre', 'mes', 
    'regiao', 'canal_aquisicao', 
    'nome_usuario', 'nome_servico'
]

# Agrupa os dados e soma o valor de cada item vendido
df_agregado = df_completo.groupby(colunas_agrupamento, as_index=False)['valor_total'].sum()

# Renomeia a coluna de valor para um nome mais claro
df_agregado.rename(columns={'valor_total': 'faturamento'}, inplace=True)

# Renomeia a coluna de região para colocar o ~
df_agregado.rename(columns={'regiao': 'região'}, inplace=True)


In [2]:
display(df_agregado)

,ano,trimestre,mes,região,canal_aquisicao,nome_usuario,nome_servico,faturamento
0,2023,4,12,Nordeste,Google,Francisco Pereira,Consultoria em Arquitetura de Software,50000.0
1,2023,4,12,Sudeste,Google,Sr. João Felipe Cunha,Alocação de Squads (Outsourcing),100000.0
2,2023,4,12,Sudeste,Google,Sr. João Felipe Cunha,Desenvolvimento de Aplicativos (Mobile/Web),100000.0
3,2024,1,3,Nordeste,Site,Mariane da Rosa,Alocação de Squads (Outsourcing),100000.0
4,2024,2,4,Centro-Oeste,Google,Mariane da Rosa,Desenvolvimento de Software Sob Medida,150000.0
...,...,...,...,...,...,...,...,...
374,2025,4,10,Sudeste,Google,Mariane da Rosa,Manutenção e Suporte de Sistemas,10000.0
375,2025,4,10,Sudeste,Meta,Alana Vieira,Alocação de Squads (Outsourcing),100000.0
376,2025,4,10,Sul,Google,Eduardo das Neves,Alocação de Squads (Outsourcing),100000.0
377,2025,4,10,Sul,Google,Eduardo das Neves,Desenvolvimento de Software Sob Medida,150000.0
